# Almgren-Chriss Optimal Execution

**Syllabus**
1. Implementation Shortfall
2. Volatility estimation

## 1. Implementation Shortfall

- **Trading trajectory**

$$ x_0 = X $$

$$ n_k = x_k - x_{k-1} $$

$$ x_k = X - \sum_{j=1}^k n_j = \sum_{j=k+1}^N n_k, \, \, \, \,\, \, \,\, \, \,   k=0,...,N $$

- **Price trajectory**

$$ S_k = S_{k-1} + \sigma \tau^{1/2} \zeta_k - \tau g(\frac{n_k}{\tau}) $$

$$ g(v) = \gamma v $$

$$ S_k = S_0 + \sigma \sum_{j=1}^k \tau^{1/2} \zeta_j - \gamma (X - x_k) $$

- **Temporary market impact**

$$ h(v) = \varepsilon sign(n_i) + \frac{\eta}{\tau} n_i $$

$$ \tilde S_k = S_{k-1} - h(\frac{n_k}{\tau}) $$

- **Implementation shortfall**

$$ IS = X S_0 - \sum_{i=1}^n n_i \tilde S_i $$

$$ \mathbb{E}(IS) = \sum_{i=1}^n \tau x_i g(\frac{n_i}{\tau}) + \sum_{i=1}^{n} n_i h(\frac{n_i}{\tau}) = \frac{1}{2} \gamma X^2 + \varepsilon \sum_{i=1}^n |n_i| + \frac{\eta-0.5\gamma \tau}{\tau} \sum_{i=1}^n n_i^2 $$

$$ \mathbb{V}(IS) = \sigma^2 \sum_{i=1}^n \tau x_i^2 $$

In [1]:
import numpy as np

def g(x, gamma):
    return gamma * x

def h(x, tau, epsilon, eta):
    """
    Returns the temporary market impact
    """
    return epsilon * np.sign(x) + eta * x / tau

def expectation(n, X=1000, tau=1, gamma=1, eta=1, eps=1):
    """
    Returns the expected IS
    n, array - Trading trajectory
    X, integer - Number of shares to be liquidated
    """
    res = 0.5*gamma*X**2 + eps*np.sum(np.abs(n)) + (eta - 0.5*gamma)/tau * np.sum(n**2)
    return res

def variance(n, X=1000, sigma=0.3, tau=1):
    res = sigma**2 * np.sum((X - n)**2)
    return res

def objective(n,risk_aversion=0.5):
    obj = expectation(n) + risk_aversion * variance(n)
    return obj

In [2]:
n_test = np.array([25, 25, 25, 25])
objective(n_test)

672462.5

In [43]:
import scipy as sp
import scipy.optimize as sco

x0 = np.zeros((20,1))
bnds = tuple((0.0, 1000) for x in range(len(x0)))
cons = ({'type': 'ie', 'fun': lambda x:  np.sum(x) <= 100})

# AAA
# AAA
opt = sco.minimize(objective, x0, method='SLSQP', bounds=bnds, constraints=cons)
opt

     fun: 1400000.0
     jac: array([-89., -89., -89., -89., -89., -89., -89., -89., -89., -89., -89.,
       -89., -89., -89., -89., -89., -89., -89., -89., -89.])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 22
     nit: 1
    njev: 1
  status: 6
 success: False
       x: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [35]:
np.sum(opt.x)

146.78742913137816

## 2. Volatility estimation